In [1]:
import numpy as np

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

train = fetch_20newsgroups(subset='train', categories=['alt.atheism', 'talk.religion.misc'])
vectorizer = CountVectorizer(stop_words="english", min_df=5)
vectors = np.asarray(vectorizer.fit_transform(train.data).todense())

In [119]:
def divide_by_features(curr_X, X, y, thres):
    X_left = X[curr_X <= thres]
    y_left = y[curr_X <= thres]
    X_right = X[curr_X > thres]
    y_right = y[curr_X > thres]

    return X_left, y_left, X_right, y_right


def entropy(c):
    entropy = 0
    for label in np.unique(c):
        prob = len(c[c == label]) / c.shape[0]
        entropy += -prob * np.log2(prob)
    return entropy


def impurity_score(y, y_left, y_right):
    parent_entropy = entropy(y)
    left_entropy = entropy(y_left)
    right_entropy = entropy(y_right)
    p = len(y_left) / len(y)
    return parent_entropy - (p * left_entropy + (1 - p) * right_entropy)


def majority_vote(y):
    print(y)
    labels = np.unique(y)
    max_cnt = 0
    max_val = 0
    for label in labels:
        cnt = len(y[y == label])
        if cnt > max_cnt:
            max_cnt = cnt
            max_val = label
    return max_val


class Node(object):
    def __init__(self, feature_name="", feature_thres="", label=-1):
        self.feature_name = feature_name
        self.feature_thres = feature_thres
        self.label = label
        self.left = None
        self.right = None


class DecisionTree(object):
    def __init__(self, max_depth=10, min_samples=2):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None

    def fit(self, X, y):
        self.root = self._fit_tree(X, y, 0)

    def _fit_tree(self, X, y, depth):
        max_score = 0
        best_features = {}
        if len(y) >= self.min_samples and depth < self.max_depth:
            for idx in range(X.shape[1]):
                curr_X = X[:, idx]
                unique_features = np.unique(curr_X)
                for val in unique_features:
                    X_left, y_left, X_right, y_right = divide_by_features(curr_X, X, y, val)
                    ig_score = impurity_score(y, y_left, y_right)

                    if ig_score > max_score:
                        max_score = ig_score
                        best_features = {
                            "feature_name": idx,
                            "feature_thres": val,
                            "left_side": (X_left, y_left),
                            "right_side": (X_right, y_right)
                        }

        if max_score > 0.:
            # print(depth, max_score, best_features)
            node = Node(best_features["feature_name"], best_features["feature_thres"])
            node.left = self._fit_tree(*best_features["left_side"], depth + 1)
            node.right = self._fit_tree(*best_features["right_side"], depth + 1)
            return node
        return Node(label=majority_vote(y))

    def predict(self, X):
        return np.array([self._predict_tree(x, self.root) for x in X])

    def _predict_tree(self, x, node):
        if node:
            # print(x, node.feature_name, node.feature_thres, node.label)
            if node.label == -1:
                val = x[node.feature_name]
                if val <= node.feature_thres:
                    return self._predict_tree(x, node.left)
                else:
                    return self._predict_tree(x, node.right)
            
            return node.label

    def print_tree(self):
        def fn(node):
            if node:
                if node.feature_name:
                    print(node.feature_name, node.feature_thres)
                else:
                    print(node.label)
                print("left -> ")
                fn(node.left)
                print("right -> ")
                fn(node.right)
            else:
                print("None end")

        fn(self.root)

In [123]:
X = np.random.randint(0, 10, (20, 7))
y = np.random.randint(0, 4, (20))
tree = DecisionTree()
tree.fit(X, y)

[2]
[3 3]
[1]
[3]
[0 0 0 0]
[3]
[0]
[3 3]
[2 2]
[0]
[2 2]
[1 1]


In [124]:
y, tree.predict(X)

(array([2, 2, 0, 0, 3, 2, 0, 0, 1, 3, 3, 3, 3, 3, 1, 1, 2, 0, 0, 2]),
 array([2, 2, 0, 0, 3, 2, 0, 0, 1, 3, 3, 3, 3, 3, 1, 1, 2, 0, 0, 2]))